In [1]:
# This program finds bad nights in a year

# The algorithm goes like this
# Create a list for stars to process, call is `stars_to_process`
# Add stars 1-1000 to stars_to_process
# Throw out variable stars from `stars_to_process`
# Throw out stars that have <80% attendance for the year from `stars_to_process`

# Create a dictionary, call it `season_mean_signal` to hold mean signals of stars for the entire data season
#   Note that the mean signal for a season for a stars is calculated from set of the internight 
#   normalized signal on each night of the season for the star.
# For each night, create a dictionary to hold data for the night, call it `nights_data_dict`
#   Create a entry in the dictionary with the night_date as key and the value as the list containing:
#   The ratio of signal_of_star_on_night[night] / season_mean_signal[star_i] for each star
# For each night, create a dict to hold std of values in `nights_data_dict`, call this dict `nights_signal_std_dict`
# (note that at this point, we have a standard deviation value for each night)
# We mark nights that have this value above a threshold (example: 0.035 for old camera, 0.03 for new camera) as bad nights for the year

In [2]:
import numpy as np
from typing import Iterable

from trout.stars import get_star, Star
from trout.stars.utils import STAR_START, STAR_END
from trout.nights.year_nights import get_nights_in_a_year

In [3]:
date_format = "%Y-%m-%d"

In [18]:
# We exclude bad stars and LPVs
stars_to_exclude = [
    1,16,41,69,82,100,115,138,166,168,195,245,255,281,285,294,317,332,
    338,356,357,366,377,410,414,441,465,466,504,533,539,592,597,600,
    628,635,664,672,685,697,703,722,736,753,755,777, 788,790, 814,824,
    842,850,852,870,877,879,888,892,904,908,912,929,950,958,981,
                   
    1007,1048,1052,1054,1065,1103,1113,1131,1143,1144,1191,1195,1197,
    1219,1223,1276,1369, 1426,1475,1495,1529,1539,1654,1687,1693,1702,
    1716,1843,1856,1873,1887,2237,2251, 2252,2502,2509,2510
                   ]
stars_to_include = list(range(1, 1000))
stars_to_include = list(filter(lambda x : x not in stars_to_exclude, stars_to_include))

In [10]:
def calculate_nights_badness_value(year : int, stars_to_use : Iterable[int], attendance_threshold : float):
    """
    Returns a dictionary of the nights with their badness value for a year by looking 
    at the data from stars in `stars_to_use`
    
    param: year: Year to analyze
    param: stars_to_user: 
        The list of stars to use for calculation of bad nights
        Note that you should not provide the variable stars in this list
    param: attendance_threshold:
        Stars with less this stated threshold will not be considered when doing
        calcualtion
    """
    all_nights = get_nights_in_a_year(year)
    
    season_mean_signal = {}
    nightly_ratio = {}
    
    # Filter out star numbers outside our stars numbers range
    stars_to_user = filter(lambda x: STAR_START <= x <= STAR_END, stars_to_use)
    
    for star in stars_to_use:
        star_data = get_star(star)
        
        # Skip the star if it doesn't pass attendance threshold
        if star_data.attendance(year) < attendance_threshold:
            continue
            
        # Select only the data for the year
        star_data.select_year(year,
            exclude_bad_nights=False, # Important to include all nights
            exclude_zeros=True
        )
        
        # Save mean for the star for the year
        season_mean_signal[star] = star_data.mean() 
        
        for _, flux, date in star_data.selected_data: # Note this is different from star_selected_data
            night_name = date.strftime(date_format)
            ratio_for_star_for_night = flux / season_mean_signal[star]
            
            # Add this ratio to the nightly_ratio
            nightly_ratio.setdefault(night_name, [ratio_for_star_for_night]).append(ratio_for_star_for_night)
    
    nightly_ratios_std = {}
    for night, ratios in nightly_ratio.items():
        ratios = np.array(ratios)
        nightly_ratios_std[night] = np.std(ratios)
        
    return nightly_ratios_std

In [11]:
def find_bad_nights(year: int, 
                    stars_to_use: Iterable[int], 
                    attendance_threshold: float, 
                    badness_threshold : float) -> Iterable[str]:
    """
    Returns the list of bad night in a given year
    param: year: Year
    param: stars_to_user: Stars to use
    param: attendance_threshold: 
        Stars with attendance less than this value (betn 0 and 1) won't be used for analysis
    param: badness_threshold:
        Nights with badness value more than the threshold will be returned
    
    return: List of bad nights
    """
    bad_nights_list = []
    badness_data = calculate_nights_badness_value(year, stars_to_use, attendance_threshold)
    for night, badness in badness_data.items():
        if badness > badness_threshold:
            bad_nights_list.append(night)
    return bad_nights_list

In [12]:
find_bad_nights(2022, stars_to_include, .80, 0.035)

['2022-09-07']

In [15]:
def get_ltpr_threshold(year:int):
    """
    Returns the LTPR aka. badness threshold to use
    for the given year
    """
    if 2003 <= year <= 2006:
        return 0.035
    elif 2007 <= year <= 2008:
        return 0.045
    else:
        return 0.035

In [17]:
years = range(2003, 2023)

for year in years:
    print(f"Bad nights for {year}")
    try:
        print(find_bad_nights(year, stars_to_include, .80, get_ltpr_threshold(year)))
    except ValueError:
        print(f"Perhaps year {year} has no data")
    print("\n\n")

Bad nights for 2003
['2003-08-12', '2003-08-29']



Bad nights for 2004
Perhaps year 2004 has no data



Bad nights for 2005
[]



Bad nights for 2006
['2006-04-04', '2006-05-06', '2006-06-29', '2006-07-06', '2006-09-05']



Bad nights for 2007
['2007-04-08', '2007-06-08', '2007-06-18', '2007-06-23', '2007-09-14']



Bad nights for 2008
['2008-02-22', '2008-02-23', '2008-03-09', '2008-03-10', '2008-04-15', '2008-05-11', '2008-05-17', '2008-05-20', '2008-06-30', '2008-07-05', '2008-08-18']



Bad nights for 2009
['2009-04-23', '2009-05-10', '2009-05-13', '2009-05-28', '2009-07-04']



Bad nights for 2010
['2010-03-17', '2010-03-20', '2010-04-27', '2010-05-26', '2010-05-27', '2010-06-28']



Bad nights for 2011
['2011-05-16', '2011-09-07']



Bad nights for 2012
['2012-05-28', '2012-07-31']



Bad nights for 2013
['2013-05-13']



Bad nights for 2014
['2014-02-17', '2014-02-18', '2014-02-22']



Bad nights for 2015
['2015-06-01', '2015-07-03']



Bad nights for 2016
['2016-05-20', '2016-